In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from collections import OrderedDict
import pickle
import pickle5
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score   
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import folium
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score, cross_val_predict
import statsmodels.api as sm

## block warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
with open("../data/ut_V6_classification-4H.pkl", "rb") as fh:
    df = pickle5.load(fh)
    
df.head()

,life_time,datetime_start,datetime_end,latitude,longitude,distance_center_km,weekday_b,charging_ports,lag1,lag2,...,hour_b_22,hour_b_23,Station_Name_BOULDER / N BOULDER REC 1,Station_Name_COMM VITALITY / 1000WALNUT,Station_Name_COMM VITALITY / 1104 SPRUCE1,4Hsplit,y_pred,Station_Name,weekday_b_name,tod
3734,118.5,2018-01-05 14:25:00,2018-01-05 16:23:30,40.000148,-105.282437,2.096847,4,2,1271.5,135.5,...,0,0,0,0,0,0,1,BOULDER / BASELINE ST1,Friday,Midday
3735,886.5,2018-01-05 17:02:00,2018-01-06 07:48:30,40.000148,-105.282437,2.096847,4,2,118.5,1271.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Friday,Afternoon
3736,326.5,2018-01-06 09:28:00,2018-01-06 14:54:30,40.000148,-105.282437,2.096847,5,2,886.5,118.5,...,0,0,0,0,0,1,0,BOULDER / BASELINE ST1,Saturday,Morning
3737,996.5,2018-01-06 17:09:00,2018-01-07 09:45:30,40.000148,-105.282437,2.096847,5,2,326.5,886.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Saturday,Afternoon
3738,52.5,2018-01-07 10:58:00,2018-01-07 11:50:30,40.000148,-105.282437,2.096847,6,2,996.5,326.5,...,0,0,0,0,0,0,0,BOULDER / BASELINE ST1,Sunday,Morning


In [3]:
df.shape

(5110, 102)

# Modeling

In [11]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))*100

In [12]:
## To test model
def test_model(y_test, y_pred):
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    #print('MAE (Mean Absolute Error):', MAE)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    #print('MSE (Mean Squared Error):', MSE)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    #print('RMSE (Root Mean Squared Error):', RMSE)
    MAPE = mean_absolute_percentage_error(y_test, y_pred)
    
    NRMSE = RMSE/np.mean(y_test)
    return MAE, RMSE, MSE, MAPE, NRMSE

In [13]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
positions = [0,1,2,3,4,5,6]

### Prepare data

In [14]:
df.head()

,life_time,datetime_start,datetime_end,latitude,longitude,distance_center_km,weekday_b,charging_ports,lag1,lag2,...,hour_b_22,hour_b_23,Station_Name_BOULDER / N BOULDER REC 1,Station_Name_COMM VITALITY / 1000WALNUT,Station_Name_COMM VITALITY / 1104 SPRUCE1,4Hsplit,y_pred,Station_Name,weekday_b_name,tod
0,118.5,2018-01-05 14:25:00,2018-01-05 16:23:30,40.000148,-105.282437,2.096847,4,2,1271.5,135.5,...,0,0,0,0,0,0,1,BOULDER / BASELINE ST1,Friday,Midday
1,886.5,2018-01-05 17:02:00,2018-01-06 07:48:30,40.000148,-105.282437,2.096847,4,2,118.5,1271.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Friday,Afternoon
2,326.5,2018-01-06 09:28:00,2018-01-06 14:54:30,40.000148,-105.282437,2.096847,5,2,886.5,118.5,...,0,0,0,0,0,1,0,BOULDER / BASELINE ST1,Saturday,Morning
3,996.5,2018-01-06 17:09:00,2018-01-07 09:45:30,40.000148,-105.282437,2.096847,5,2,326.5,886.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Saturday,Afternoon
4,52.5,2018-01-07 10:58:00,2018-01-07 11:50:30,40.000148,-105.282437,2.096847,6,2,996.5,326.5,...,0,0,0,0,0,0,0,BOULDER / BASELINE ST1,Sunday,Morning


To make coding easier the names of the models are changed:
- M1: Baseline
- M5: Full

In [33]:
## M1
features1 = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
             'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
             'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday',
             'tod_Evening','tod_Midday', 'tod_Morning',
             'Station_Name_BOULDER / N BOULDER REC 1',
             'Station_Name_COMM VITALITY / 1000WALNUT',
             'Station_Name_COMM VITALITY / 1104 SPRUCE1'] # dow + tod
## M5
features5_cat = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
                 'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
                 'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday',
                 'tod_Evening','tod_Midday', 'tod_Morning',
                 'Station_Name_BOULDER / N BOULDER REC 1',
                 'Station_Name_COMM VITALITY / 1000WALNUT',
                 'Station_Name_COMM VITALITY / 1104 SPRUCE1']
features5_con = ['lag1', 'lag2', 'lag3','near_charge_time_4H', 'near_charge_energy_4H',
                 'charge_time_4H', 'charge_energy_4H']

Test train split

In [34]:
split = 0.8

## Quantile regression - Gradiant boosting

#### Get/split data

In [35]:
train, test = np.split(df, [int(split * len(df))])
print("Training shape:",train.shape)
print("Testing shape:", test.shape)

Training shape: (4069, 102)
Testing shape: (1018, 102)


In [36]:
trainS = train[train['y_pred'] == 0]
trainL = train[train['y_pred'] == 1]

testS = test[test['y_pred'] == 0]
testL = test[test['y_pred'] == 1]

In [37]:
print("SHORT: Train: {} Test: {}".format(trainS.shape[0], testS.shape[0]))
print("LONG: Train: {} Test: {}".format(trainL.shape[0], testL.shape[0]))

SHORT: Train: 2430 Test: 628
LONG: Train: 1639 Test: 390


In [38]:
## Get X - train
# Baseline
X1_S_train = trainS[features1]
X1_L_train = trainL[features1]
# Full
X5_S_train = trainS[features5_cat+features5_con]
scaler = StandardScaler()
X5_S_train[features5_con] = scaler.fit_transform(X5_S_train[features5_con])
X5_S_test = testS[features5_cat+features5_con]
X5_S_test[features5_con] = scaler.transform(X5_S_test[features5_con])

X5_L_train = trainL[features5_cat+features5_con]
scaler = StandardScaler()
X5_L_train[features5_con] = scaler.fit_transform(X5_L_train[features5_con])
X5_L_test = testL[features5_cat+features5_con]
X5_L_test[features5_con] = scaler.transform(X5_L_test[features5_con])

## Get y - train
y_S_train = trainS['life_time']
y_L_train = trainL['life_time']

## Get y - test
y_S_test = testS['life_time']
y_L_test = testL['life_time']

Data frames for quantile predictions:

In [40]:
quantile_labels = ['5th','25th','50th','75th','95th']
# M1 - Baseline
qr_M1_S = pd.DataFrame(columns=quantile_labels+['true'])
qr_M1_S['true'] = y_S_test
qr_M1_L = pd.DataFrame(columns=quantile_labels+['true'])
qr_M1_L['true'] = y_L_test

# M5 - Full
qr_M5_S = pd.DataFrame(columns=quantile_labels+['true'])
qr_M5_S['true'] = y_S_test
qr_M5_L = pd.DataFrame(columns=quantile_labels+['true'])
qr_M5_L['true'] = y_L_test

In [41]:
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]

In [42]:
results = {}

#### Train M1

In [43]:
## SHORT
for i in range(len(quantiles)):
    reg = sm.QuantReg(y_S_train, X1_S_train).fit(q=quantiles[i])
    pred = reg.predict(X1_S_test)
    qr_M1_S[quantile_labels[i]] = pred

## LONG
for i in range(len(quantiles)):
    reg = sm.QuantReg(y_L_train, X1_L_train).fit(q=quantiles[i])
    pred = reg.predict(X1_L_test)
    qr_M1_L[quantile_labels[i]] = pred

In [44]:
M1_pred_com = qr_M1_S.append(qr_M1_L)

In [45]:
results['M1'] = {}

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M1_S['true'],qr_M1_S['50th'])
results['M1']['S'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M1_S['true'],qr_M1_S['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M1_L['true'],qr_M1_L['50th'])
results['M1']['L'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M1_L['true'],qr_M1_L['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(M1_pred_com['true'],M1_pred_com['50th'])
results['M1']['Com'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(M1_pred_com['true'],M1_pred_com['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

In [46]:
results['M1']

{'S': {'RMSE_test': 423.2860157907176,
  'MAE_test': 239.16042407800978,
  'rsq_test': -0.1325552842718325,
  'MAPE_test': 939.7899392357056,
  'NRMSE_test': 1.5069024393810269},
 'L': {'RMSE_test': 536.1426331864948,
  'MAE_test': 380.2271521456975,
  'rsq_test': -0.5157405894082854,
  'MAPE_test': 1169.5071297606603,
  'NRMSE_test': 0.7230264248784832},
 'Com': {'RMSE_test': 469.7368951193659,
  'MAE_test': 293.20366960492356,
  'rsq_test': -0.0011862375860149932,
  'MAPE_test': 1027.7955426784683,
  'NRMSE_test': 1.0270472213890376}}

#### Train M5

In [47]:
## SHORT
for i in range(len(quantiles)):
    reg = sm.QuantReg(y_S_train, X5_S_train).fit(q=quantiles[i])
    pred = reg.predict(X5_S_test)
    qr_M5_S[quantile_labels[i]] = pred

## LONG
for i in range(len(quantiles)):
    reg = sm.QuantReg(y_L_train, X5_L_train).fit(q=quantiles[i])
    pred = reg.predict(X5_L_test)
    qr_M5_L[quantile_labels[i]] = pred

In [48]:
M5_pred_com = qr_M5_S.append(qr_M5_L)

In [49]:
results['M5'] = {}

## Get metrics
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M5_S['true'],qr_M5_S['50th'])
results['M5']['S'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M5_S['true'],qr_M5_S['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M5_L['true'],qr_M5_L['50th'])
results['M5']['L'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M5_L['true'],qr_M5_L['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(M5_pred_com['true'],M5_pred_com['50th'])
results['M5']['Com'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(M5_pred_com['true'],M5_pred_com['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

In [50]:
results['M5']

{'S': {'RMSE_test': 422.25484646286964,
  'MAE_test': 239.49540474281895,
  'rsq_test': -0.12704395756948972,
  'MAPE_test': 911.0387502292032,
  'NRMSE_test': 1.5032314662858106},
 'L': {'RMSE_test': 538.5147322810175,
  'MAE_test': 378.2359721278066,
  'rsq_test': -0.5291826869320275,
  'MAPE_test': 1197.0375462150405,
  'NRMSE_test': 0.7262253690056772},
 'Com': {'RMSE_test': 470.20366689455903,
  'MAE_test': 292.64748851506374,
  'rsq_test': -0.0031769590731474118,
  'MAPE_test': 1020.6060689271172,
  'NRMSE_test': 1.0280677855808562}}

### Evaluate QR fit

In [51]:
def ICP(q1, q2, true):
    if (true >= q1) and (true <= q2):
        return 1
    else:
        return 0

In [52]:
######### M1 #########
####### SHORT #######
## 5%-95%
qr_M1_S['MIL5-95'] = qr_M1_S['95th']-qr_M1_S['5th']
qr_M1_S = qr_M1_S.reset_index()
# Get ICP
temp = []
for i in range(len(y_S_test)):
    row = qr_M1_S.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
qr_M1_S['ICP5-95'] = temp

## 25%-75%
qr_M1_S['MIL25-75'] = qr_M1_S['75th']-qr_M1_S['25th']
# Get ICP
temp = []
for i in range(len(y_S_test)):
    row = qr_M1_S.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
qr_M1_S['ICP25-75'] = temp

####### LONG #######
## 5%-95%
qr_M1_L['MIL5-95'] = qr_M1_L['95th']-qr_M1_L['5th']
qr_M1_L = qr_M1_L.reset_index()
# Get ICP
temp = []
for i in range(len(y_L_test)):
    row = qr_M1_L.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
qr_M1_L['ICP5-95'] = temp

## 25%-75%
qr_M1_L['MIL25-75'] = qr_M1_L['75th']-qr_M1_L['25th']
# Get ICP
temp = []
for i in range(len(y_L_test)):
    row = qr_M1_L.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
qr_M1_L['ICP25-75'] = temp

####### COMBINED #######
## 5%-95%
M1_pred_com['MIL5-95'] = M1_pred_com['95th']-M1_pred_com['5th']
M1_pred_com = M1_pred_com.reset_index()
# Get ICP
temp = []
for i in range(len(M1_pred_com)):
    row = M1_pred_com.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
M1_pred_com['ICP5-95'] = temp

## 25%-75%
M1_pred_com['MIL25-75'] = M1_pred_com['75th']-M1_pred_com['25th']
# Get ICP
temp = []
for i in range(len(M1_pred_com)):
    row = M1_pred_com.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
M1_pred_com['ICP25-75'] = temp


In [53]:
######### M5 #########
####### SHORT #######
## 5%-95%
qr_M5_S['MIL5-95'] = qr_M5_S['95th']-qr_M5_S['5th']
qr_M5_S = qr_M5_S.reset_index()
# Get ICP
temp = []
for i in range(len(y_S_test)):
    row = qr_M5_S.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
qr_M5_S['ICP5-95'] = temp

## 25%-75%
qr_M5_S['MIL25-75'] = qr_M5_S['75th']-qr_M5_S['25th']
# Get ICP
temp = []
for i in range(len(y_S_test)):
    row = qr_M5_S.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
qr_M5_S['ICP25-75'] = temp

####### LONG #######
## 5%-95%
qr_M5_L['MIL5-95'] = qr_M5_L['95th']-qr_M5_L['5th']
qr_M5_L = qr_M5_L.reset_index()
# Get ICP
temp = []
for i in range(len(y_L_test)):
    row = qr_M5_L.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
qr_M5_L['ICP5-95'] = temp

## 25%-75%
qr_M5_L['MIL25-75'] = qr_M5_L['75th']-qr_M5_L['25th']
# Get ICP
temp = []
for i in range(len(y_L_test)):
    row = qr_M5_L.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
qr_M5_L['ICP25-75'] = temp

####### COMBINED #######
## 5%-95%
M5_pred_com['MIL5-95'] = M5_pred_com['95th']-M5_pred_com['5th']
M5_pred_com = M5_pred_com.reset_index()
# Get ICP
temp = []
for i in range(len(M5_pred_com)):
    row = M5_pred_com.loc[i]
    temp.append(ICP(row['5th'], row['95th'], row['true']))
M5_pred_com['ICP5-95'] = temp

## 25%-75%
M5_pred_com['MIL25-75'] = M5_pred_com['75th']-M5_pred_com['25th']
# Get ICP
temp = []
for i in range(len(M5_pred_com)):
    row = M5_pred_com.loc[i]
    temp.append(ICP(row['25th'], row['75th'], row['true']))
M5_pred_com['ICP25-75'] = temp


In [54]:
qr_M1_S = qr_M1_S.set_index('index')
qr_M1_L = qr_M1_L.set_index('index')
M1_pred_com = M1_pred_com.set_index('index')
qr_M5_S = qr_M5_S.set_index('index')
qr_M5_L = qr_M5_L.set_index('index')
M5_pred_com = M5_pred_com.set_index('index')

In [55]:
qr_M1_S = qr_M1_S.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
qr_M1_L = qr_M1_L.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
M1_pred_com = M1_pred_com.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
qr_M5_S = qr_M5_S.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
qr_M5_L = qr_M5_L.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)
M5_pred_com = M5_pred_com.merge(df[['tod','weekday_b_name','Station_Name']],left_index=True,right_index=True)

In [56]:
print("Baseline - SHORT")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = qr_M1_S[qr_M1_S['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))
    
print("\nBaseline - LONG")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = qr_M1_L[qr_M1_L['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))

print("\nBaseline - COMBINED")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = M1_pred_com[M1_pred_com['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))

Baseline - SHORT
Morning:
 MIL(5-95): 1113.0 ICP(5-95): 0.877   MIL(25-75): 323.0 ICP(25-75): 0.526
Midday:
 MIL(5-95): 1264.0 ICP(5-95): 0.847   MIL(25-75): 303.0 ICP(25-75): 0.463
Afternoon:
 MIL(5-95): 1096.0 ICP(5-95): 0.814   MIL(25-75): 490.0 ICP(25-75): 0.436
Evening:
 MIL(5-95): nan ICP(5-95): nan   MIL(25-75): nan ICP(25-75): nan

Baseline - LONG
Morning:
 MIL(5-95): 1466.0 ICP(5-95): 0.778   MIL(25-75): 912.0 ICP(25-75): 0.5
Midday:
 MIL(5-95): 2181.0 ICP(5-95): 0.796   MIL(25-75): 1050.0 ICP(25-75): 0.265
Afternoon:
 MIL(5-95): 1057.0 ICP(5-95): 0.58   MIL(25-75): 586.0 ICP(25-75): 0.188
Evening:
 MIL(5-95): 1916.0 ICP(5-95): 0.902   MIL(25-75): 335.0 ICP(25-75): 0.528

Baseline - COMBINED
Morning:
 MIL(5-95): 1161.0 ICP(5-95): 0.864   MIL(25-75): 403.0 ICP(25-75): 0.523
Midday:
 MIL(5-95): 1395.0 ICP(5-95): 0.84   MIL(25-75): 410.0 ICP(25-75): 0.434
Afternoon:
 MIL(5-95): 1086.0 ICP(5-95): 0.758   MIL(25-75): 513.0 ICP(25-75): 0.377
Evening:
 MIL(5-95): 1916.0 ICP(5-95): 0.

In [57]:
print("Full - SHORT")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = qr_M5_S[qr_M1_S['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))
    
print("\nFull - LONG")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = qr_M5_L[qr_M5_L['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))

print("\nFull - COMBINED")
for t in ['Morning', 'Midday', 'Afternoon', 'Evening']:
    res = M5_pred_com[M5_pred_com['tod'] == t]
    print("{}:\n MIL(5-95): {} ICP(5-95): {}   MIL(25-75): {} ICP(25-75): {}".format(t,round(np.mean(res['MIL5-95']),0),
                                                  round(np.mean(res['ICP5-95']),3),
                                                  round(np.mean(res['MIL25-75']),0),
                                                  round(np.mean(res['ICP25-75']),3)))

Full - SHORT
Morning:
 MIL(5-95): 1115.0 ICP(5-95): 0.877   MIL(25-75): 335.0 ICP(25-75): 0.509
Midday:
 MIL(5-95): 1248.0 ICP(5-95): 0.847   MIL(25-75): 295.0 ICP(25-75): 0.459
Afternoon:
 MIL(5-95): 1110.0 ICP(5-95): 0.836   MIL(25-75): 496.0 ICP(25-75): 0.45
Evening:
 MIL(5-95): nan ICP(5-95): nan   MIL(25-75): nan ICP(25-75): nan

Full - LONG
Morning:
 MIL(5-95): 1532.0 ICP(5-95): 0.833   MIL(25-75): 917.0 ICP(25-75): 0.444
Midday:
 MIL(5-95): 2062.0 ICP(5-95): 0.816   MIL(25-75): 1034.0 ICP(25-75): 0.265
Afternoon:
 MIL(5-95): 1095.0 ICP(5-95): 0.667   MIL(25-75): 538.0 ICP(25-75): 0.13
Evening:
 MIL(5-95): 1901.0 ICP(5-95): 0.917   MIL(25-75): 374.0 ICP(25-75): 0.508

Full - COMBINED
Morning:
 MIL(5-95): 1172.0 ICP(5-95): 0.871   MIL(25-75): 415.0 ICP(25-75): 0.5
Midday:
 MIL(5-95): 1364.0 ICP(5-95): 0.843   MIL(25-75): 401.0 ICP(25-75): 0.431
Afternoon:
 MIL(5-95): 1106.0 ICP(5-95): 0.796   MIL(25-75): 506.0 ICP(25-75): 0.374
Evening:
 MIL(5-95): 1901.0 ICP(5-95): 0.917   MIL(25

In [65]:
## print("------ SHORT ------ ")
print("Base (5%-95%): MIL: {} ICP: {}".format(round(np.mean(qr_M1_S['MIL5-95']),2), 
                                            round(np.sum(qr_M1_S['ICP5-95'])/len(qr_M1_S),3)))
print("Full (5%-95%): MIL: {} ICP: {}".format(round(np.mean(qr_M5_S['MIL5-95']),2), 
                                            round(np.sum(qr_M5_S['ICP5-95'])/len(qr_M5_S),3)))
print("\n")
print("Base (25%-75%): MIL: {} ICP: {}".format(round(np.mean(qr_M1_S['MIL25-75']),2), 
                                            round(np.sum(qr_M1_S['ICP25-75'])/len(qr_M1_S),3)))
print("Full (25%-75%): MIL: {} ICP: {}".format(round(np.mean(qr_M5_S['MIL25-75']),2), 
                                            round(np.sum(qr_M5_S['ICP25-75'])/len(qr_M5_S),3)))


print("\n------ LONG ------ ")
print("Base (5%-95%): MIL: {} ICP: {}".format(round(np.mean(qr_M1_L['MIL5-95']),2), 
                                            round(np.sum(qr_M1_L['ICP5-95'])/len(qr_M1_L),3)))
print("Full (5%-95%): MIL: {} ICP: {}".format(round(np.mean(qr_M5_L['MIL5-95']),2), 
                                            round(np.sum(qr_M5_L['ICP5-95'])/len(qr_M5_L),3)))
print("\n")
print("Base (25%-75%): MIL: {} ICP: {}".format(round(np.mean(qr_M1_L['MIL25-75']),2), 
                                            round(np.sum(qr_M1_L['ICP25-75'])/len(qr_M1_L),3)))
print("Full (25%-75%): MIL: {} ICP: {}".format(round(np.mean(qr_M5_L['MIL25-75']),2), 
                                            round(np.sum(qr_M5_L['ICP25-75'])/len(qr_M5_L),3)))

print("\n------ COMBIEND ------ ")
print("Base (5%-95%): MIL: {} ICP: {}".format(round(np.mean(M1_pred_com['MIL5-95']),2), 
                                            round(np.sum(M1_pred_com['ICP5-95'])/len(M1_pred_com),3)))
print("Full (5%-95%): MIL: {} ICP: {}".format(round(np.mean(M5_pred_com['MIL5-95']),2), 
                                            round(np.sum(M5_pred_com['ICP5-95'])/len(M5_pred_com),3)))
print("\n")
print("Base (25%-75%): MIL: {} ICP: {}".format(round(np.mean(M1_pred_com['MIL25-75']),2), 
                                            round(np.sum(M1_pred_com['ICP25-75'])/len(M1_pred_com),3)))
print("Full (25%-75%): MIL: {} ICP: {}".format(round(np.mean(M5_pred_com['MIL25-75']),2), 
                                            round(np.sum(M5_pred_com['ICP25-75'])/len(M5_pred_com),3)))

Base (5%-95%): MIL: 1177.78 ICP: 0.841
Full (5%-95%): MIL: 1175.34 ICP: 0.849


Base (25%-75%): MIL: 372.4 ICP: 0.465
Full (25%-75%): MIL: 373.0 ICP: 0.465

------ LONG ------ 
Base (5%-95%): MIL: 1776.46 ICP: 0.826
Full (5%-95%): MIL: 1761.34 ICP: 0.856


Base (25%-75%): MIL: 496.16 ICP: 0.433
Full (25%-75%): MIL: 511.22 ICP: 0.408

------ COMBIEND ------ 
Base (5%-95%): MIL: 1407.14 ICP: 0.835
Full (5%-95%): MIL: 1399.84 ICP: 0.852


Base (25%-75%): MIL: 419.81 ICP: 0.453
Full (25%-75%): MIL: 425.96 ICP: 0.443


In [59]:
results = {}
results['M1'] = {}
results['M5'] = {}

MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M1_S['true'],qr_M1_S['50th'])
results['M1']['S'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M1_S['true'],qr_M1_S['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M1_L['true'],qr_M1_L['50th'])
results['M1']['L'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M1_L['true'],qr_M1_L['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(M1_pred_com['true'],M1_pred_com['50th'])
results['M1']['Com'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(M1_pred_com['true'],M1_pred_com['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M5_S['true'],qr_M5_S['50th'])
results['M5']['S'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M5_S['true'],qr_M5_S['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(qr_M5_L['true'],qr_M5_L['50th'])
results['M5']['L'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(qr_M5_L['true'],qr_M5_L['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(M5_pred_com['true'],M5_pred_com['50th'])
results['M5']['Com'] = {'RMSE_test':RMSE_test,'MAE_test':MAE_test,
                      'rsq_test':r2_score(M5_pred_com['true'],M5_pred_com['50th']),
                      'MAPE_test':MAPE_test,'NRMSE_test':NRMSE_test}

In [60]:
for m in ['M1','M5']:
    #print(m)
    for l in ['S','L','Com']:
        r = results[m][l]
        RMSE = round(r['RMSE_test'],0)
        R2 = round(r['rsq_test'],2)
        MAE = round(r['MAE_test'],0)
        MAPE = round(r['MAPE_test'],0)
        NRMSE = round(r['NRMSE_test'],3)
        
        print("{} - {}:\n R2: {} RMSE: {} MAE: {} MAPE: {} NRMSE: {}".format(m,l,R2,RMSE,MAE,MAPE,NRMSE))  
    print("\n")

M1 - S:
 R2: -0.13 RMSE: 423.0 MAE: 239.0 MAPE: 940.0 NRMSE: 1.507
M1 - L:
 R2: -0.52 RMSE: 536.0 MAE: 380.0 MAPE: 1170.0 NRMSE: 0.723
M1 - Com:
 R2: -0.0 RMSE: 470.0 MAE: 293.0 MAPE: 1028.0 NRMSE: 1.027


M5 - S:
 R2: -0.13 RMSE: 422.0 MAE: 239.0 MAPE: 911.0 NRMSE: 1.503
M5 - L:
 R2: -0.53 RMSE: 539.0 MAE: 378.0 MAPE: 1197.0 NRMSE: 0.726
M5 - Com:
 R2: -0.0 RMSE: 470.0 MAE: 293.0 MAPE: 1021.0 NRMSE: 1.028


